In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install torch

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.nn.functional as F

In [5]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/FineTunedModels/')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/FineTunedModels/')

In [6]:
completetweets = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/completetweets.csv')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [14]:
data = []
label_mapping = {
    0: "Supporter",
    1: "Against",
    2: "Manipulator",
    3: "Neutral"
}


In [9]:
completetweets.head(5)

,Source.Name,Datetime,Tweet Id,Text,Username,retweetCount,likeCount,quoteCount,coordinates,mentionedUsers,retweetedTweet,hashtags,viewCount,TweetID,LanguageUsed,TranslatedText
0,April25.csv,25/04/2023 19:24,1.650000e+18,#TheKerlaStory,SArDArDCPATEL,0,4,0,NaN,NaN,NaN,['TheKerlaStory'],191.0,1.650000e+18,en,#TheKerlaStory
1,April26.csv,26/04/2023 19:19,1.650000e+18,Just Watched The Trailer of #Thekerlastory 😬\n...,HaqSeReview,9,41,0,NaN,"[User(username='adah_sharma', id=295714480, di...",NaN,['Thekerlastory'],1972.0,1.650000e+18,hi,Just watched the trailer of #thekerlastory 😬\n...
2,April26.csv,26/04/2023 19:14,1.650000e+18,@Starboy2079 #mustwatcgthekerlastory #Thekerla...,Saurabh317m,0,0,0,NaN,"[User(username='Starboy2079', id=1515651214200...",NaN,"['mustwatcgthekerlastory', 'Thekerlastory']",1270.0,1.650000e+18,en,@Starboy2079 #mustwatcgthekerlastory #Thekerla...
3,April26.csv,26/04/2023 19:11,1.650000e+18,#Thekerlastory 👍🏻👍🏻🩷🩷,AryanSh31941116,0,0,0,NaN,NaN,NaN,['Thekerlastory'],105.0,1.650000e+18,en,#Thekerlastory 👍🏻👍🏻🩷🩷
4,April26.csv,26/04/2023 18:19,1.650000e+18,#TheKerlaStory @ARanganathan72\nhttps://t.co/5...,Rabindra_Sun,0,0,0,NaN,"[User(username='ARanganathan72', id=1344897174...",NaN,['TheKerlaStory'],167.0,1.650000e+18,en,#TheKerlaStory @ARanganathan72\nhttps://t.co/5...


In [15]:
# Loop through unseen tweets
for index,row in completetweets.iterrows():
  text = row['TranslatedText']
  date = row['Datetime']
    # Tokenize the tweet and convert to model input format
  inputs = tokenizer.encode_plus(text, add_special_tokens = True,max_length=256,pad_to_max_length=True,return_attention_mask=True,return_tensors = 'pt')
  inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU

    # Make a prediction
  with torch.no_grad():
    outputs = model(**inputs)

    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits).item()
    predicted_label_text = label_mapping[predicted_label]

    # Map the predicted label to its corresponding textual label
    #predicted_label_text = label_mapping[predicted_label]

    # Append the data to the list
    data.append({
        'Date': date,
        'Tweet': text,
        'Predicted Label': predicted_label_text
    })

# Create a DataFrame from the data
df = pd.DataFrame(data)

In [16]:
df.head()

,Date,Tweet,Predicted Label
0,25/04/2023 19:24,#TheKerlaStory,Manipulator
1,26/04/2023 19:19,Just watched the trailer of #thekerlastory 😬\n...,Supporter
2,26/04/2023 19:14,@Starboy2079 #mustwatcgthekerlastory #Thekerla...,Manipulator
3,26/04/2023 19:11,#Thekerlastory 👍🏻👍🏻🩷🩷,Supporter
4,26/04/2023 18:19,#TheKerlaStory @ARanganathan72\nhttps://t.co/5...,Neutral


In [17]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Final_Predictions.csv')